<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTunning_Testing_For_AviationQADataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tunning: https://github.com/frank-morales2020/Cloud_curious/blob/master/FineTuning_Mistral_7b_hfdeployment_dataset_AviationQA.ipynb

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
#!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


!pip install huggingface_hub -q

In [2]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

peft_model_id = "frankmorales2020/Mistral-7B-v0.1_AviationQA"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id, use_fast=True)

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# load into pipeline
pipe = pipeline("document-question-answering", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")

dataset = load_dataset("sakharamg/AviationQA")
#dataset = dataset.map(create_prompt_formats)

# Convert dataset to OAI messages
#dataset = dataset.map(create_conversation)

#dataset = dataset.map(create_prompt_formats)

# save datasets to disk
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [10]:
dataset

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 10807
})

In [11]:
nrec=10
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]
#dataset_final_Messages=dataset['messages'][0:nrec]

In [12]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer

In [13]:
datasetF

,id,Question,Answer
0,a4905077116,What is the name of the Engine Manufacturer of...,Lycoming
1,a7510929859,When was the Last Flight Review or Equivalent ...,"May 13, 2017"
2,a5430892651,What was the flight conducted under for the ac...,Part 91: General aviation - Personal
3,a9405153070,What was the Observation Time of the accident ...,13:54 Local
4,a8193000066,What is the name of the Engine Manufacturer of...,LYCOMING
5,a9610719319,What is the Airworthiness Certificate of the a...,Normal
6,a7001234003,Was there any Operating Certificate(s) held of...,None
7,a9135502144,How many Seats are there in the aircraft beari...,2
8,a157434574,What is the Serial Number of the aircraft bear...,07-12-03
9,a1523521076,What is the Airframe Total Time of the acciden...,10663 Hrs as of last inspection


In [14]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

In [15]:
eval_dataset

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 10807
})

In [16]:
nrec=3297
eval_dataset[nrec]

{'id': 'a9981762110',
 'Question': 'Define the event of the accident no. GAA16CA528?',
 'Answer': 'Loss of control on ground'}

In [17]:
eval_dataset[nrec]['Question']

'Define the event of the accident no. GAA16CA528?'

In [18]:
eval_dataset[nrec]['Answer']

'Loss of control on ground'

https://nbviewer.org/github/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb

In [19]:
from random import randint
rand_idx = randint(0, len(eval_dataset))

#rand_idx = 4071
#,3600, 5570,10077 ## good answer

In [20]:
rand_idx

2385

In [21]:
#rand_idx = randint(0, len(eval_dataset))
#modified_conversation = [
#    {"role": "user", "content": eval_dataset[rand_idx]['Question']},
#    {"role": "assistant", "content": eval_dataset[rand_idx]['Answer']}
#]

In [ ]:
# Inspect the structure of the messages
#print(eval_dataset[rand_idx]["messages"][:2])


modified_conversation = [
    {"role": "user", "content": eval_dataset[rand_idx]['Question'] if eval_dataset[rand_idx]['Question'] is not None else ""}, # Handle potential None values
    {"role": "assistant", "content": eval_dataset[rand_idx]['Answer'] if eval_dataset[rand_idx]['Answer'] is not None else ""}  # Handle potential None values
]

#print(modified_conversation)

#Use the chat template to format the input for the language model
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map

#prompt = generation_pipeline.tokenizer.apply_chat_template(modified_conversation, tokenize=False, add_generation_prompt=True)

prompt = eval_dataset[rand_idx]['Question']
outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

In [ ]:
outputs

In [24]:
#rand_idx = randint(0, len(eval_dataset))
print(f"Question:\n{eval_dataset[rand_idx]['Question']}")
print()
print(f"Original Answer:\n{eval_dataset[rand_idx]['Answer']}")
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

Question:
What is the age of the pilot who is responsible for the accdent no. ERA14LA008?

Original Answer:
28

Generated Answer:
43,Male


In [25]:
prompt='capital of russia?'

outputs = generation_pipeline(prompt, max_new_tokens=256, do_sample=True, temperature=0.7,
                              top_k=50, top_p=0.9, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [26]:
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")

Generated Answer:
Moscow


In [27]:
q=eval_dataset[1]['Question']

In [28]:
#eval_dataset.shuffle().select(range(number_of_eval_samples))[9]

In [29]:
len(eval_dataset)

10807

In [ ]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/test_dataset_AviationQA.json", split="train")

def evaluate(sample):

    prompt = sample['Question']
    #print(prompt)

    #prompt = eval_dataset[rand_idx]['Question']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)
    #prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    #outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt)+8:].strip()

    if predicted_answer == sample['Answer']:
          #print(f"Question:\n{sample['Question']}")
          #print(f"Original Answer:\n{sample['Answer']}")
          #print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt)+8:].strip()}")
          return 1
    else:
          return 0

success_rate = []
number_of_eval_samples = 10000
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

 99%|█████████▉| 9941/10000 [2:59:03<00:50,  1.17it/s]

In [ ]:
print(f"Accuracy (Eval dataset and predict): {accuracy*100:.2f}%")